In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf

# import my plotting stuff

import sys
sys.path.append('./PythonHelpers/')
import PlotLib.Hist1D as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)
def fitfunc_gauss_constBKG(x, A, mu, sigma, b):
    return A * np.exp(-0.5*((x-mu)/sigma)**2) + b

In [ ]:
loadEvents = 100000
runIDs = arr([637,635,639,633,638,636,640,634])
# runIDs = arr([634])

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, loadEvents*4)
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],loadEvents)

# runs[runID]["data"][evt,pixel,M[key]]
M = runs[runIDs[0]]["M"]
scalefactor = arr([1,100000/58975,1,1,1,1,1,1])

M.keys()

map_pixToAx = [2,0,3,1] # pix 00 -> bottom left -> ax[2]


In [ ]:
# 2d: Amplitude vs Risetime

runIDs = arr([637,635,639,633,638,636,640,634])

# rangeX = [5, 26]
rangeX = [5,20]
rangeY = [50, 185]
# binNX = 40
binNX = 40
binNY = 20
NameX = "Risetime"
NameY = "Amplitude"
logScale = False

risetime_cut_width = 0.212 * 3 # 1 sigma = 0.212 ns
risetime_regions = { # only keep this to show difference between the cuts done by eye (these here) and the automatic ones (that are used later one)
    637: [[6.,8],[6.,8],[6.,8],[6.,8]],
    635: [[6.,8.5],[6.,8.5],[6.,8.5],[6.,8.5]],
    639: [[5.5,7.5],[6.,8.5],[6.,8],[6.,8.5]],
    633: [[5.5,7.5],[6.,8],[5.5,7.5],[6.,8]],
    638: [[5.5,7.5],[5.5,8],[5.5,7.5],[5.5,8]],
    636: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    640: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    634: [[5.5,7],[5.5,7.7],[5.5,7.3],[5.5,7.7]],
}

pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])

for runID in runIDs:
    FileName = f"Amplitude_vs_Risetime_{runs[runID]["krum_trim"]}DAC"
    fig, axs = Plot.create_fig(4,4, sharex=False, sharey=False, figsize=[6,4.5], width_ratios=[1,.2,1,.2], height_ratios=[.2,1,.2,1], flatten=False)        
        
    for i_row in range(2):
        for i_col in range(2):
            i_pix = i_row*2+i_col
            i_pix = pixMap[i_pix]
            mask = [True if x > 0 else False for x in runs[runID]["data"][:,i_pix,M[NameX]]]
            mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)
            entriesX = runs[runID]["data"][mask,i_pix,M[NameX]]
            entriesY = arr(runs[runID]["data"][mask,i_pix,M[NameY]])*1000
            hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])
            
            # draw 2d histogram
            extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]
            axs[2*i_row+1, 2*i_col].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
            
            # find highest bin, define cut region, draw vlines
            maxbin = np.unravel_index(np.argmax(hist, axis=None), hist.shape)   
            maxbin_risetime = (binsx[maxbin[0]]+binsx[maxbin[0]+1])/2
            
            runs[runID]["risetime_cut"] = arr([maxbin_risetime-risetime_cut_width, maxbin_risetime+risetime_cut_width])
            
            axs[2*i_row+1, 2*i_col].axvline(runs[runID]["risetime_cut"][0], color="C1", lw=.5, ls="--")
            axs[2*i_row+1, 2*i_col].axvline(runs[runID]["risetime_cut"][1], color="C1", lw=.5, ls="--")
            
            # axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][0], color="red", lw=.5, ls="--")
            # axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][1], color="red", lw=.5, ls="--")
        
            # draw y-projection
            hist1d, bins = np.histogram(entriesY, bins=binNY, range=rangeY)
            axs[2*i_row+1, 2*i_col+1].set_xlim(0, 1.3*np.max(hist1d))
            axs[2*i_row+1, 2*i_col+1].set_ylim(rangeY)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal", alpha=1, fill=False)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal",alpha=0.2, fill=True)
    
            # draw x-projection
            hist1d, bins = np.histogram(entriesX, bins=binNX, range=rangeX)
            axs[2*i_row, 2*i_col].set_xlim(rangeX)
            axs[2*i_row, 2*i_col].set_ylim(0, 1.3*np.max(hist1d))
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=1, fill=False)
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=0.2, fill=True)

            title_str = PixNames[i_pix]+"\nN={:.1f}k".format(np.sum(hist)/1000)
            axs[2*i_row+1, 2*i_col].text(0.5, 0.95, title_str, transform=axs[2*i_row+1, 2*i_col].transAxes, ha="center", fontdict={"color":"white"}, va="top")

    # Hist.finalize(runs[runID], fig, axs.flatten(), title=NameY+" vs "+NameX, xlabel=None, ylabel=None, subplots_adjust=[0.,0.,0.93])
    Plot.finalize(runs[runID], fig, axs.flatten(),
        NameY+" vs "+NameX + "\nrisetime cuts shown", None, None, None,
        title_linebreak=True, ER1=False,
        legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=3.5,
        param_dict={"ER1Param":True, "thr":None, "run":False, "sample":True, "fontsize":"medium"},
        grid=False, logy=False, subplots_adjust=None, labelpad=1.)
    fig.subplots_adjust(wspace=0., hspace=0)
    for ax in axs.flatten():
        ax.grid(False)
        ax.tick_params("both", direction="in", top=True, right=True)
    for i_row in range(3):
        for i_col in range(4):
            axs[i_row, i_col].set_xticklabels([])
            axs[i_col, i_row+1].set_yticklabels([])
    for i_row in range(2):
        for i_col in range(2):
            axs[2*i_row, 2*i_col].axis("off")
            axs[2*i_row, 2*i_col+1].axis("off")
            axs[2*i_row+1, 2*i_col+1].axis("off")
    for i in range(2):
        axs[2*i+1,0].set_ylabel("Amplitude / mV", loc="top")
        axs[-1, 2*i].set_xlabel("Risetime / ns", loc="right")
    
    Plot.savefig(fig, "ChargeCalibration/"+FileName)

In [ ]:
# krum_trim wise: Amplitude (& fit). without risetime cuts

binRange = arr([0.025,0.2])*1000
binN = 40
binWidth = (binRange[1]-binRange[0])/binN
key = "AmplitudeEst"
Filename = "Amplitude_krumTrimWise_preCut"
runIDs = arr([637,635,639,633,638,636,640,634])

binWidth = (binRange[1]-binRange[0])/binN
fitRangeBinsLow = arr([
    [27,26,24,28],
    [26,25,23,26],
    [25,23,22,24],
    [24,21,20,23],
    [22,21,17,23],
    [22,20,16,22],
    [21,19,16,21],
    [21,18,15,20],
])
fitRangeBinsHigh = arr([ [35]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [38]*4 ])

for i in range(8):
    fitRangeBinsLow[i] = arr([fitRangeBinsLow[i,map_pixToAx[j]] for j in range(4)]) 

fig, axs = Plot.create_fig(2,4, [8,10])
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i,runID in enumerate(runIDs):
    runs[runID]["charge_cal_preCut"] = uarr([0]*4, [0]*4)
    for i_pix in range(4):

        mask = np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2 # remove events with more than 1 pixel fired (decided 2025-03-11)
        entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        fitMask = np.zeros(len(bins)-1, dtype=bool)
        for j in range(len(bins)-1):
            if j > fitRangeBinsLow[i,i_pix] and j < fitRangeBinsHigh[i,i_pix]:
                fitMask[j] = True
        
        p0 = arr([1000, 130, 10])
        pBounds = ([0, 100, 0], [5000, 200, 100])
        popt, perr, chi2, ndeg = Hist.curve_fit_wrapper(hist, bins, fitMask, fitfunc_gauss, p0=p0, maxfev=100000, bounds=pBounds)
        
        runs[runID]["charge_cal_preCut"][i_pix] = 1617 / uf(popt[1], popt[2]) *1000 # *1000 to convert back to V
        
        fitX = np.arange(bins[:-1][fitMask][0], bins[:-1][fitMask][-1]+binWidth, binWidth/20)
        x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     
    
        axs[i].plot(fitX, fitfunc_gauss(fitX,*popt), color="black", lw=1, label="")
        axs[i].plot(x, fitfunc_gauss(x,*popt), color="black", lw=.5, ls=":", label="")

        if False:
            axs[i].axvline(popt[1]+popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
            axs[i].axvline(popt[1]-popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
        
        # label = PixNames[i_pix]+"; N="+str(int(np.sum(hist)/1000))+"k" +"\n  "+r"$\mu$"+"={:.1u}".format(uf(popt[1], perr[1])) +"\n  "+r"chi2red"+"={:.1f} / {:n}".format(chi2,ndeg)
        label = PixNames[i_pix]
        Plot.draw(axs[i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
    axs[i].set_xlim(binRange)

axs[0].set_ylim(0,axs[0].get_ylim()[1]*1.2)
axs[0].legend(loc="upper left", fontsize="medium", borderaxespad=3.5, frameon=False)

subtitles = dict_to_arr(runs, runIDs, "krum_trim")
subtitles = ["Krummenacher\ntrimming = "+str(subtitles[i])+" DAC" for i in range(len(subtitles))]

Plot.finalize(runs[runID], fig, axs,
    "Waveform amplitude (pre risetime cut)", "Waveform amplitude / mV", f"Entries / {binWidth:.1f} mV", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=3.5,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
# krum_trim wise: Amplitude (& fit). with risetime cuts

binRange = arr([0.025,0.2])*1000
binN = 50
binWidth = (binRange[1]-binRange[0])/binN
key = "AmplitudeEst"
Filename = "Amplitude_krumTrimWise_postCut"
runIDs = arr([637,635,639,633,638,636,640,634])

binWidth = (binRange[1]-binRange[0])/binN


fig, axs = Plot.create_fig(2,4, [8,10])
# pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i,runID in enumerate(runIDs):
    runs[runID]["charge_cal"] = uarr([0]*4, [0]*4)
    for i_pix in range(4):
        # mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_regions[runID][i_pix][0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_regions[runID][i_pix][1]) # old risetime cuts (done by eye)
        mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > runs[runID]["risetime_cut"][0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < runs[runID]["risetime_cut"][1]) # automatic risetime cuts
        mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)

        entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        fitMask = hist > 0.22*np.max(hist)
        
        p0 = arr([np.max(hist), 160, 10])
        pBounds = ([0, 100, 0], [10*np.max(hist), 200, 100])
        popt, perr, chi2, ndeg = Hist.curve_fit_wrapper(hist, bins, fitMask, fitfunc_gauss, p0=p0, maxfev=100000, bounds=pBounds)
        
        runs[runID]["charge_cal"][i_pix] = 1617 / uf(popt[1], popt[2]) * 1000 # *1000 to convert back to V
        
        fitX = np.arange(bins[:-1][fitMask][0], bins[:-1][fitMask][-1]+binWidth, binWidth/20)
        x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     
    
        axs[i].plot(fitX, fitfunc_gauss(fitX,*popt), color="black", lw=1, label="")
        axs[i].plot(x, fitfunc_gauss(x,*popt), color="black", lw=.5, ls=":", label="")

        if False:
            axs[i].axvline(popt[1]+popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
            axs[i].axvline(popt[1]-popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
        
        label = PixNames[i_pix]
        Plot.draw(axs[i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
    axs[i].set_xlim(binRange)

axs[0].set_ylim(0,axs[0].get_ylim()[1]*1.2)  

subtitles = dict_to_arr(runs, runIDs, "krum_trim")
subtitles = ["Krummenacher\ntrimming = "+str(subtitles[i])+" DAC" for i in range(len(subtitles))]

axs[0].legend(loc="upper left", fontsize="medium", borderaxespad=3.5, frameon=False)

Plot.finalize(runs[runID], fig, axs,
    "Waveform amplitude (post risetime cut)", "Waveform amplitude / V", f"Entries / {binWidth*1000:.1f} mV", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=3.5,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
# (pix-wise) Fe55 peak position (before/after risetime cuts)
Filename = "PeakPos_pixWise"
runIDs = arr([637,635,639,633,638,636,640,634])

fig, axs = Plot.create_fig(2,2,[6,4.5])
for i_pix in range(4):
    i_ax = map_pixToAx[i_pix]
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_trim_nominal")
    
    labels = {
        "charge_cal_preCut": "before risetime-cut",
        "charge_cal": "after risetime-cut" }
    colors = {
        "charge_cal_preCut": "black",
        "charge_cal": "C"+str(i_pix) }
    alphas = {
        "charge_cal_preCut": 0.4,
        "charge_cal": 1 }
    for Key in ["charge_cal_preCut","charge_cal"]:
        charge_calS = uarr([val(runs[runID][Key][i_pix]) for runID in runIDs],[dev(runs[runID][Key][i_pix]) for runID in runIDs])
        charge_calS = 1000 / (charge_calS / 1617)
        axs[i_ax].errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color=colors[Key], label=labels[Key], capsize=3, markersize=3, alpha=alphas[Key])
    
    axs[i_ax].set_xticks(krum_bias_trimS, dict_to_arr(runs, runIDs, "krum_trim"))
    title = PixNames[i_pix]
    # axs[i_ax].set_title(title)
    
axs[0].set_ylim(axs[0].get_ylim()[0],axs[0].get_ylim()[1]*1.05)
axs[0].legend(loc="lower left", fontsize="medium", borderaxespad=2, frameon=False)
    
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, axs,
    r"Fe55 $K_\alpha$ Peak Position and Width", "krum_trimming [DAC]", r"$K_\alpha$ Peak Position [mV]", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc=None, legend_fontdict={"size":"medium"}, legend_borderaxespad=2.5,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
Filename = "Amplitude_pixWise_preCut"

fig, axs = Plot.create_fig(2,2,[6,4.5])
# runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,639,636,634])
binRange = [25,190]
binN = 40
krum_bias_trims = dict_to_arr(runs, runIDs, "krum_trim_nominal")
colors = arr(Plot.get_color_range(krum_bias_trims, maxLightness=0.8))[::-1]
key = "Amplitude"
# colors = Hist.get_color_range(len(runIDs), maxLightness=0.8)

for i_runID, runID in enumerate(runIDs):
    mask = np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2 # remove events with more than 1 pixel fired (decided 2025-03-11)
    
    for i_pix in range(4):
        entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        label = "krum trim "+str(runs[runID]["krum_trim"])+" DAC"
        i_ax = map_pixToAx[i_pix]
        Plot.draw(axs[i_ax], hist, bins, color=colors[i_runID], label=label, fill_alpha=0.1)
        Fe55peak = 1 / (runs[runID]["charge_cal_preCut"][i_pix].nominal_value / 1620) * 1000
        axs[i_ax].axvline(Fe55peak, color=colors[i_runID], ls="--", alpha=0.7)

axs[0].set_xlim(binRange)
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, axs,
    r"Fe55 Amplitudes (pre risetime-cut)", "Amplitude [mV]", r"Entries", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc="upper left", legend_fontdict={"size":"small"}, legend_borderaxespad=1.7,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
Filename = "Amplitude_pixWise_postCut"

fig, axs = Plot.create_fig(2,2,[6,4.5])
# runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,639,636,634])
binRange = [25,190]
binN = 40
krum_bias_trims = dict_to_arr(runs, runIDs, "krum_trim_nominal")
colors = arr(Plot.get_color_range(krum_bias_trims, maxLightness=0.8))[::-1]
key = "Amplitude"

for i_runID, runID in enumerate(runIDs):
    
    for i_pix in range(4):
        mask = (runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_regions[runID][i_pix][0]) & (runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_regions[runID][i_pix][1])
        mask = np.logical_and(mask, np.count_nonzero(runs[runID]["data"][:,:,M["Charge"]], axis=1) < 2) # remove events with more than 1 pixel fired (decided 2025-03-11)
        entries = arr(runs[runID]["data"][mask,i_pix,M[key]])*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        label = "krum trim "+str(runs[runID]["krum_trim"])+" DAC"
        i_ax = map_pixToAx[i_pix]
        Plot.draw(axs[i_ax], hist, bins, color=colors[i_runID], label=label, fill_alpha=0.1)
        Fe55peak = 1 / (runs[runID]["charge_cal"][i_pix].nominal_value / 1620) * 1000
        axs[i_ax].axvline(Fe55peak, color=colors[i_runID], ls="--", alpha=0.7)

axs[0].set_xlim(binRange)
subtitles = ["Pix 0,1", "Pix 1,1", "Pix 0,0", "Pix 1,0"]
Plot.finalize(runs[runID], fig, axs,
    r"Fe55 Amplitudes (post risetime-cut)", "Amplitude [mV]", r"Entries", subtitles,
    title_linebreak=True, ER1=False,
    legend_loc="upper left", legend_fontdict={"size":"small"}, legend_borderaxespad=1.7,
    param_dict={"ER1Param":False, "run":False, "sample":True, "fontsize":"medium"},
    grid=False, logy=False, subplots_adjust=None, labelpad=1.)
Plot.savefig(fig, "ChargeCalibration/"+Filename)

In [ ]:
if False:
    fig, ax = Hist.create_fig(1,1,[6,4])

    runIDs = arr([637,635,639,633,638,636,640,634])
    for i_pix in range(4):
        krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_bias_trim")
        charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
        # Hist.draw(axS, val(charge_cal), np.arange(len(runIDs)), color="C"+str(i_pix), label=PixNames[i_pix], fill_alpha=0.1)
        ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3)

    ax.set_xticks(krum_bias_trimS)
    Hist.finalize(runs[runID], fig, ax, "krum_bias_trim [nA]", r"Calibration [$e^{-}/$ V]", "Charge Calibration Factors", subtitles, measurement="Fe55_all", logy=False)
    fig.savefig("output/ChargeCalibration/CalibrationFactors.pdf")
    fig.savefig("output/ChargeCalibration/png/CalibrationFactors.png", dpi=400)

In [ ]:
# (single plot) Fe55 peak position
Filename = "PeakPos"
runIDs = arr([637,635,639,633,638,636,640,634])

fig, ax = Hist.create_fig(1,1,[6,4])

for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_trim_nominal")
    charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    charge_calS = 1 / (charge_calS / 1620) * 1000
    ax.plot(krum_bias_trimS-0.15+(i_pix*0.1), val(charge_calS), color="C"+str(i_pix),markersize=3, zorder=10, alpha=0.4)
    ax.errorbar(krum_bias_trimS-0.15+(i_pix*0.1), val(charge_calS), yerr=dev(charge_calS), fmt="o", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3, zorder=11)

ax.set_xticks(krum_bias_trimS,dict_to_arr(runs, runIDs, "krum_trim"))
Hist.finalize(runs[runID], fig, ax, "krum_trimming [DAC]", "Fe55 Peak Position [mV]", "Fe55 Peak Position\n(after cutting on risetime)", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/"+Filename+".pdf")
fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/png/"+Filename+".png", dpi=400)

In [ ]:
def get_calibration_factors(runs_, krum_trims):
    if len(krum_trims) != 4:
        print("Error: krum_trims must have length 4, coresponding to the 4 pixels")\
    
    cal_runIDs = arr([637,635,639,633,638,636,640,634])
    cal_krum_trims = dict_to_arr(runs_, cal_runIDs, "krum_trim")
    
    cal_factors = np.zeros(4)
    for i_pix in range(4):
        cal_runID = cal_runIDs[np.where(krum_trims[i_pix] == cal_krum_trims)[0][0]]
        factors[i_pix] = runs_[cal_runID]["charge_cal"][i_pix].nominal_value
    return factors

# load TB runs
loadEvents = 10
runIDs = arr([184,190,192])

if not "runs" in globals():
    runs = {}
    
for runID in runIDs:
    runs[runID] = load_run(runID, loadEvents*4)
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],loadEvents)    

# get calibraton factors
for runID in runIDs:
    print(str(runID)+": ", end="")
    krum_trims = runs[runID]["krum_trim"]
    
    factors = get_calibration_factors(runs, krum_trims)
    for factor in factors:
        print("{:.2f}".format(factor), end=", ")
    print("")

In [ ]:
# Amplitude - before/after risetime cut

binRange = arr([0.025,0.2]) * 1000
binN = 40
key = "AmplitudeEst"
Filename = "Amplitude_risetimeCut"
runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,634])

fig, axs = Hist.create_fig(2,len(runIDs), [8,6])
pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i,runID in enumerate(runIDs):
    runs[runID]["charge_cal"] = uarr([0]*4, [0]*4)
    for i_pix in range(4):
        # before risetime cut
        entries = runs[runID]["data"][:,i_pix,M[key]]*1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        label = PixNames[i_pix]+" N={:.1f}k".format(np.sum(hist)/1000)
        Hist.draw(axs[2*i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
        title = "BEFORE risetime-cut (krum_trimming = "+str(runs[runID]["krum_trim"])+" DAC)"
        axs[2*i].set_title(title, fontsize=9)
        
        # after risetime cut
        risetime_region = risetime_regions[runID][i_pix]
        mask1 = runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_region[0]
        mask2 = runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_region[1]
        mask = np.logical_and(mask1, mask2)

        entries = runs[runID]["data"][:,i_pix,M[key]][mask] * 1000
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        label = PixNames[i_pix]+" N={:.1f}k".format(np.sum(hist)/1000)
        Hist.draw(axs[2*i+1], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
        
        title = "AFTER risetime cut (krum_trimming = "+str(runs[runID]["krum_trim"])+" DAC)"
        axs[2*i+1].set_title(title, fontsize=9)
    axs[i].set_xlim(binRange)

Hist.finalize(runs[runID], fig, axs, "Amplitude [mV]", "Entries", "Amplitude\n(before/after risetime cut)", subtitles, measurement="Fe55_all", logy=False)
fig.subplots_adjust(hspace=.15, wspace=.09, top=0.87)

fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/"+Filename+".pdf")
fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/png/"+Filename+".png", dpi=400)